In [84]:
from MySQLdb import connect
import witxcraft as wc
from dask.distributed import Client

SINGLESTORE_HOST = "172.17.0.1"
SINGLESTORE_PORT = 3306

SINGLESTORE_URI = "mysql+mysqldb://root:root@{}:{}/demo".format(SINGLESTORE_HOST, SINGLESTORE_PORT)

In [85]:
client = Client()
client.cluster

/usr/local/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44801 instead
  warnings.warn(


LocalCluster(1d72c504, 'tcp://127.0.0.1:42065', workers=4, threads=8, memory=15.43 GiB)

In [86]:
import dask.dataframe as dd

df = dd.read_sql_table('comments_small', SINGLESTORE_URI, npartitions=8, index_col='id')
df

/usr/local/lib/python3.9/site-packages/dask/dataframe/io/sql.py:113: SAWarning: Unknown schema content: '  KEY `__UNORDERED` () USING CLUSTERED COLUMNSTORE'
  table = sa.Table(table, m, autoload=True, autoload_with=engine, schema=schema)
/usr/local/lib/python3.9/site-packages/dask/dataframe/io/sql.py:113: SAWarning: Unknown schema content: '  , SHARD KEY () '
  table = sa.Table(table, m, autoload=True, autoload_with=engine, schema=schema)


,text,creation_date,score
npartitions=8,,,
53275096.0,object,datetime64[ns],int64
53510777.0,...,...,...
...,...,...,...
54924863.0,...,...,...
55160544.0,...,...,...


In [87]:
len(df)

1000

In [88]:
ml = wc.fromwasmmod('example_wasm.wasm', witx='iface.witx')
ml.sentiment('Welcome home')

Generating "/tmp/tmphkenp8s2/bindings.py"


compound    0.458831
positive    0.750000
negative    0.000000
neutral     0.250000
dtype: float64

In [89]:
out = df['text'].apply(ml.sentiment, meta=ml.sentiment.ret_meta)
out.compute()

,compound,positive,negative,neutral
id,,,,
53350749,-0.046426,0.000000,0.000000,1.000000
53351044,0.458831,0.062500,0.000000,0.937500
53392176,0.688815,0.314965,0.078362,0.606673
53392204,0.000000,0.000000,0.000000,1.000000
53392716,0.361158,0.128205,0.000000,0.871795
...,...,...,...,...
55143070,0.397948,0.211356,0.000000,0.788644
55143087,-0.046426,0.000000,0.000000,1.000000
55143393,0.345516,0.148154,0.131096,0.720750


In [102]:
from pymysql import connect
s2 = connect(host=SINGLESTORE_HOST, port=SINGLESTORE_PORT, user='root', passwd='root', db='demo', local_infile=True)
cursor = s2.cursor()

In [98]:
cursor.execute("select count(*) from comments")
cursor.fetchall()

((82037742,),)

In [99]:
cursor.execute("""
    create or replace function sentiment_vec(input varchar(256) not null)
    returns table(
        compound double,
        positive double,
        negative double,
        neutral double
    ) as wasm infile '/workspaces/wasi-data-python/examples/example_wasm.wasm';
""")

1

In [104]:
cursor.execute("select * from sentiment_vec('I love wasm')")
cursor.fetchall()

((0.6369499429264264, 0.8076923076923077, 0.0, 0.1923076923076923),)

In [112]:
cursor.execute("select c.score, c.text, s.* from (select * from comments limit 100) as c, sentiment_vec(c.text) as s")
for row in cursor.fetchall():
    print("-----------------------------------------------------------------------------------------------")
    print(row[1])
    print("score {} sentiment compount {}".format(row[0], row[2]))
    print("-----------------------------------------------------------------------------------------------")

-----------------------------------------------------------------------------------------------
[Rule of 3/5/0](https://en.cppreference.com/w/cpp/language/rule_of_three) for `printer` is broken.
score 4 sentiment compount -0.4766576055745744
-----------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------
Something like this should work: `allow write: if request.resource.data.message.matches('[a-z0-9-_\*#]+')`. If you're having a hard time making that work, edit your question to show what you tried already.
score 4 sentiment compount 0.4588314677411235
-----------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------
Where is the `test_string` coming from?
score 4 sentiment compount -0.04642568745829288
----------------------------------

In [106]:
cursor.fetchall()

((4, 0.5573704017131537, 0.36507936507936506, 0.0, 0.6349206349206349),
 (4, 0.0, 0.0, 0.0, 1.0),
 (5, -0.40192382526938203, 0.0, 0.08466819221967964, 0.9153318077803203),
 (5, 0.4465681526609422, 0.13372543655678656, 0.0, 0.8662745634432134),
 (6,
  -0.09764669773479735,
  0.0601909506019095,
  0.06807804068078041,
  0.8717310087173101),
 (5, 0.401923825269382, 0.10112359550561799, 0.0, 0.898876404494382),
 (4,
  -0.11364112532772533,
  0.02970407801749054,
  0.03796453411400549,
  0.932331387868504),
 (4, 0.401923825269382, 0.2783505154639176, 0.0, 0.7216494845360826),
 (9, 0.0, 0.0, 0.0, 1.0),
 (4, -0.6248933269389456, 0.0, 0.5604395604395604, 0.43956043956043955),
 (4, 0.2914477694178259, 0.05863367401828941, 0.0, 0.9413663259817105),
 (4,
  -0.268376068314099,
  0.08704482808646452,
  0.1392233667005174,
  0.773731805213018),
 (4, -0.04642568745829288, 0.0, 0.0, 1.0),
 (5, -0.5267415375673765, 0.0, 0.20560747663551404, 0.7943925233644861),
 (4, 0.0, 0.0, 0.0, 1.0),
 (4, 0.31821099